In [1]:
# 라이브러리 설치
!pip install gdrive_dataset

In [2]:
from gdrivedataset import loader

file_id = "1IbyL1762M0Lgoq7_qm2RaXfl3ney0e9i"
loader.load_from_google_drive(file_id)

========== files ============

data/dataset



In [3]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
import os

In [4]:
DATA_DIR = 'data/dataset'

In [5]:
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [6]:
# 결측치 확인
data.isna().sum()

id          0
document    0
label       0
dtype: int64

In [7]:
# train test split
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [8]:
train

,index,id,document,label
0,1750,1751,내가 이 영화를 100% 다 소화시킬수 없다는 것에 정말 슬프다...,1
1,4257,4258,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,0
2,4899,4900,한마디만 한다. 보지마. 이런건 보는거 아니여,0
3,4583,4584,솔직히.. 지금 보기엔 많이 유치하지.. ㅋㅋ 그당시니까 대박소리나오는거지,0
4,612,613,명화를 기름종이에 대고 그린듯하다!하지만 나 역시 이렇게 만들었을 것이다,0
...,...,...,...,...
3745,4707,4708,"전형적인b급.어정쩡한스토리,구성이며 퀄리티의차이를극복하진못한다.",0
3746,1411,1412,지루하다...재미없다... 시간만 아깝다..,0
3747,4810,4811,최고진짜대박겨울만대면보고진짜돌려보기안좋아하는데도열번도넘게본드라마,1
3748,2960,2961,쓰레기더ㅏㅋㅋㅋㅋㅋㅋㅋㅋㅋ돈주고봣엇는데내돈돌려내,0


In [9]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","")
train[:5]

,index,id,document,label,preprocessed
0,1750,1751,내가 이 영화를 100% 다 소화시킬수 없다는 것에 정말 슬프다...,1,내가 이 영화를 다 소화시킬수 없다는 것에 정말 슬프다
1,4257,4258,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,0,너무 졸립다 ㅋ 수면제도 아니고 ㅋ
2,4899,4900,한마디만 한다. 보지마. 이런건 보는거 아니여,0,한마디만 한다 보지마 이런건 보는거 아니여
3,4583,4584,솔직히.. 지금 보기엔 많이 유치하지.. ㅋㅋ 그당시니까 대박소리나오는거지,0,솔직히 지금 보기엔 많이 유치하지 ㅋㅋ 그당시니까 대박소리나오는거지
4,612,613,명화를 기름종이에 대고 그린듯하다!하지만 나 역시 이렇게 만들었을 것이다,0,명화를 기름종이에 대고 그린듯하다하지만 나 역시 이렇게 만들었을 것이다


In [10]:
train['preprocessed'] = train['preprocessed'].str.replace(" +"," ")
train[:5]

,index,id,document,label,preprocessed
0,1750,1751,내가 이 영화를 100% 다 소화시킬수 없다는 것에 정말 슬프다...,1,내가 이 영화를 다 소화시킬수 없다는 것에 정말 슬프다
1,4257,4258,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,0,너무 졸립다 ㅋ 수면제도 아니고 ㅋ
2,4899,4900,한마디만 한다. 보지마. 이런건 보는거 아니여,0,한마디만 한다 보지마 이런건 보는거 아니여
3,4583,4584,솔직히.. 지금 보기엔 많이 유치하지.. ㅋㅋ 그당시니까 대박소리나오는거지,0,솔직히 지금 보기엔 많이 유치하지 ㅋㅋ 그당시니까 대박소리나오는거지
4,612,613,명화를 기름종이에 대고 그린듯하다!하지만 나 역시 이렇게 만들었을 것이다,0,명화를 기름종이에 대고 그린듯하다하지만 나 역시 이렇게 만들었을 것이다


In [11]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 6.1 MB/s 
     |████████████████████████████████| 448 kB 63.1 MB/s 


In [12]:
from konlpy.tag import Okt

In [13]:
han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [14]:
tokenized = []
for sentence in train['preprocessed']:
  tokens = okt.morphs(sentence, stem=True)
  tokenize = " ".join(tokens)
  tokenized.append(tokenize)
train['tokenized_stem'] = pd.DataFrame(tokenized)

train.head()

,index,id,document,label,preprocessed,tokenized_stem
0,1750,1751,내가 이 영화를 100% 다 소화시킬수 없다는 것에 정말 슬프다...,1,내가 이 영화를 다 소화시킬수 없다는 것에 정말 슬프다,내 가 이 영화 를 다 소화 시키다 없다 것 에 정말 슬프다
1,4257,4258,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,0,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,너무 졸리다 ㅋ 수면제 도 아니다 ㅋ
2,4899,4900,한마디만 한다. 보지마. 이런건 보는거 아니여,0,한마디만 한다 보지마 이런건 보는거 아니여,한마디 만 하다 보다 이렇다 보다 아니다
3,4583,4584,솔직히.. 지금 보기엔 많이 유치하지.. ㅋㅋ 그당시니까 대박소리나오는거지,0,솔직히 지금 보기엔 많이 유치하지 ㅋㅋ 그당시니까 대박소리나오는거지,솔직하다 지금 보기 엔 많이 유치하다 ㅋㅋ 그 당시 니까 대박 소리 나 오는 거지
4,612,613,명화를 기름종이에 대고 그린듯하다!하지만 나 역시 이렇게 만들었을 것이다,0,명화를 기름종이에 대고 그린듯하다하지만 나 역시 이렇게 만들었을 것이다,명화 를 기름종이 에 대고 그리다 하다 나 역시 이렇게 만들다 것 이다


In [15]:
main_pos = []
for sentence in train['document']:
  pos = okt.pos(sentence)
  main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
  main_words_str = " ".join(main_words)
  main_pos.append(main_words_str)
train['main_pos'] = pd.DataFrame(main_pos)

train.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,1750,1751,내가 이 영화를 100% 다 소화시킬수 없다는 것에 정말 슬프다...,1,내가 이 영화를 다 소화시킬수 없다는 것에 정말 슬프다,내 가 이 영화 를 다 소화 시키다 없다 것 에 정말 슬프다,내 이 영화 다 소화 시킬수 없다는 것 정말 슬프다
1,4257,4258,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,0,너무 졸립다 ㅋ 수면제도 아니고 ㅋ,너무 졸리다 ㅋ 수면제 도 아니다 ㅋ,너무 졸립다 수면제 아니고
2,4899,4900,한마디만 한다. 보지마. 이런건 보는거 아니여,0,한마디만 한다 보지마 이런건 보는거 아니여,한마디 만 하다 보다 이렇다 보다 아니다,한마디 한다 보지마 이런건 보는거 아니여
3,4583,4584,솔직히.. 지금 보기엔 많이 유치하지.. ㅋㅋ 그당시니까 대박소리나오는거지,0,솔직히 지금 보기엔 많이 유치하지 ㅋㅋ 그당시니까 대박소리나오는거지,솔직하다 지금 보기 엔 많이 유치하다 ㅋㅋ 그 당시 니까 대박 소리 나 오는 거지,솔직히 지금 보기 많이 유치하지 당시 대박 소리 거지
4,612,613,명화를 기름종이에 대고 그린듯하다!하지만 나 역시 이렇게 만들었을 것이다,0,명화를 기름종이에 대고 그린듯하다하지만 나 역시 이렇게 만들었을 것이다,명화 를 기름종이 에 대고 그리다 하다 나 역시 이렇게 만들다 것 이다,명화 기름종이 대고 그린듯 하다 나 역시 이렇게 만들었을 것


In [16]:
X_train = train.main_pos
y_train = train.label

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)

In [18]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vec, y_train)

LogisticRegression()

In [19]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3447,3448,스토리는 천군보다 나은데....연출이나 연기 참 못한다...,0,스토리는 천군보다 나은데연출이나 연기 참 못한다,스토리 는 천군 보다 낫다 연출 이나 연기 차다 못 하다,스토리 천군 나은데 연출 연기 참 한다
1,3203,3204,단체로 낚시하나ㅡㅡ??? 진짜 재미없음,0,단체로 낚시하나ㅡㅡ 진짜 재미없음,단체 로 낚시 하나 ㅡㅡ 진짜 재미없다,단체 낚시 하나 진짜 재미없음
2,412,413,케이지! 졸리! 액션! 아무런 조화를 이루지 못하고 그냥 질주한다,0,케이지 졸리 액션 아무런 조화를 이루지 못하고 그냥 질주한다,케이 지 졸리 액션 아무렇다 조화 를 이루다 못 하고 그냥 질주 한 다,케이 졸리 액션 아무런 조화 이루지 못 그냥 질주 다
3,3742,3743,너무 재미있어서 두번이나 반복해서 봤네...,1,너무 재미있어서 두번이나 반복해서 봤네,너무 재미있다 두 번 이나 반복 하다 보다,너무 재미있어서 번 반복 해서 봤네
4,3405,3406,김수현이 시나리오 썼나.. 완전 개막장.. 재미도 별로.,0,김수현이 시나리오 썼나 완전 개막장 재미도 별로,김수현 이 시나리오 써다 완전 개막 장 재미 도 별로,김수현 시나리오 썼나 완전 개막 재미 별로


In [20]:
X_val = val.main_pos
y_val = val.label

X_val_vec = vectorizer.transform(X_val)

In [21]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[0 0 0 ... 0 1 1]


In [22]:
from sklearn import metrics
print("accuracy = ", metrics.accuracy_score(y_val, y_pred))

accuracy =  0.8264


In [24]:
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


In [25]:
test.isna().sum()

id          0
document    0
dtype: int64

In [26]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [28]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]


In [29]:
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [30]:
submission['label'] = pred_test

In [31]:
submission.head()

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1


In [32]:
submission.to_csv("submission_baseline2.csv", index=False)